# Group Project Proposal: *insert name of project*

**Group Number:** 24

**Group Members:** Sienna Muller, Pascal Lo, Matthew Haryanto, Juan Pablo Villalobos

### Introduction: *Pascal*

*what to include:*

- Provide some relevant background information on the topic so that someone unfamiliar with it will be prepared to understand the rest of your proposal
- Clearly state the question you will try to answer with your project
- Identify and describe the dataset that will be used to answer the question

**Question:** Predicting the genre using a knn classification algorithm based on opening weekend profits, film budget, and profitability.

### Preliminary Exploratory Data Analysis: *Matthew*

*what to include:*

- Demonstrate that the dataset can be read from the web into R 
- Clean and wrangle your data into a tidy format
- Using only training data, summarize the data in at least one table (this is exploratory data analysis). An example of a useful table could be one that reports the number of observations in each class, the means of the predictor variables you plan to use in your analysis and how many rows have missing data. 
- Using only training data, visualize the data with at least one plot relevant to the analysis you plan to do (this is exploratory data analysis). An example of a useful visualization could be one that compares the distributions of each of the predictor variables you plan to use in your analysis.

In [9]:
# below are all of the libraries that must be imported for this project

library(tidyverse)
library(rvest) #not sure if we need this one
library(repr)
library(tidymodels)


In [ ]:
# read dataset from the csv

url <- "https://raw.githubusercontent.com/reisanar/datasets/master/HollywoodMovies.csv"

hollywood_movies <- read_csv(url)

#rough code to check all the different genres
hollywood_movies|>
group_by(Genre)|>
summarize(DomesticGross = sum(DomesticGross))


Rows: 970 Columns: 16
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (4): Movie, LeadStudio, Story, Genre
dbl (12): RottenTomatoes, AudienceScore, TheatersOpenWeek, OpeningWeekend, B...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


Genre,n
<chr>,<int>
Action,166
Adventure,30
Animation,51
Biography,14
Comedy,177
Crime,15
Documentary,7
Drama,109
Fantasy,6


In [12]:
#split into training and testing data

hollywood_split <- initial_split(hollywood_movies, prop = 0.75, strata = Genre)
hollywood_training <- training(hollywood_split)
hollywood_testing <- testing(hollywood_split)

In [24]:
# clean/tidy data

In [ ]:
# summarize the data in at least one table

In [ ]:
# visualize with one plot relevant to planned analysis

### Methods: *Pablo*

*what to include:*

- Explain how you will conduct either your data analysis and which variables/columns you will use. Note - you do not need to use all variables/columns that exist in the raw data set. In fact, that's often not a good idea. For each variable think: is this a useful variable for prediction?
- Describe at least one way that you will visualize the results

grouping genres by:
- action/adventure
- Romance/Drama
- Mystery/crime
- Horror/thriller
- biography/documentary 

### Expected Outcomes and Significance: *Sienna*

*what to include:*
- What do you expect to find?
- What impact could such findings have?
- What future questions could this lead to?